## Package Loading

In [1]:
import tensorflow as tf
import numpy as np
import konlpy as knp
import random as rand
import codecs
import os
import string
from collections import Counter

/home/jsu/.pyenv/versions/anaconda3-5.0.1/envs/t/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Data Preparation

In [2]:
data_path = './dataset'
file_paths = []

In [3]:
for cat in os.listdir(data_path):
    for file_path in os.listdir(data_path+'/'+cat):
        file_paths.append(data_path+'/{}/{}'.format(cat, file_path))

In [4]:
file_paths

['./dataset/Coat/Coat_Kinds.txt',
 './dataset/Coat/Coat_Define.txt',
 './dataset/Coat/Coat_Character.txt',
 './dataset/Coat/Coat_Kinds2.txt',
 './dataset/New Balance/NewBalance_tribia.txt',
 './dataset/New Balance/NewBalance_index.txt',
 './dataset/Hood/Hood_Character.txt',
 './dataset/Hood/Hood_Define.txt',
 './dataset/Suit/Suit_Man.txt',
 './dataset/Suit/Suit_Quality.txt',
 './dataset/Suit/Suit_Rule.txt',
 './dataset/Suit/Suit_Women.txt',
 './dataset/Suit/Suit_Define.txt',
 './dataset/etc/fuCKing.txt',
 './dataset/T-shirts/T-shirts_History.txt',
 './dataset/T-shirts/T-shirts_Kinds.txt',
 './dataset/RunningShoes/RunningShoes_Brand.txt',
 './dataset/RunningShoes/RunningShoes_Usage.txt',
 './dataset/RunningShoes/RunningShoes_ByFashion.txt',
 './dataset/RunningShoes/RunningShoes_History.txt',
 './dataset/RunningShoes/RunningShoes_etc.txt',
 './dataset/RunningShoes/RunningShoes_index.txt',
 './dataset/Adidas/Adidas_Index.txt',
 './dataset/Adidas/Adidas_Active.txt',
 './dataset/Adidas/Adid

## Data Preprocessing - Words
1. Punctuation
2. Postposition
3. Blankspace
4. []
5. Freq, Unique, Subsampling

In [5]:
class InvaderTools(object):
    kor_exept = ['에서', '부터', '으로', '이다', '있다', '없다', '이나', '만큼', '만큼이나', '하다', '한다', '해서', '두면',
                 '보다', '에도', '이었', '기에', '거나', '나도', '들이', '은', '는', '을', '를', '이', '가', '의', '에',
                 '랑', '와']
    
    @staticmethod
    def char_trans(text, trans_dict):
        processed = ''
        
        for char in text:
            processed += trans_dict.get(char, char)
            
        return processed
    
    @staticmethod
    def replace_trans(text, trans_dict):
        processed = text  #* Immutable
        
        for key in trans_dict.keys():
            processed = processed.replace(key, trans_dict[key])
            
        return processed
    
    @staticmethod
    def remove(text, *trans_lists):  #* **, *
        trans_dict = {word: '' for trans_list in trans_lists for word in trans_list}
        return InvaderTools.replace_trans(text, trans_dict)

In [6]:
words = []

In [7]:
for path in file_paths:
    file = codecs.open(path, 'r', encoding='utf-16')
    processed = [InvaderTools.remove(token, string.punctuation, string.digits) for line in file.readlines() for token in line.split('\n') if token != '']
    words.extend([word for doc in processed for word in doc.split()])

In [8]:
kkma = knp.tag.Kkma()

In [9]:
processed_words = [result for word in words for result in kkma.nouns(word) if result != []]

In [10]:
print(processed_words)

['싱글', '코트', '정면', '때', '세로', '한', '코트', '더블', '코트', '정면', '때', '세로', '두', '코트', '요즘', '바깥쪽', '한', '줄', '한', '줄', '페이크', '보통', '라펠', '손목', '손목턴업', '턴', '업', '등', '부분', '액션플릿', '폴로', '코트', '스터', '코트', '원단', '사용', '코트', '서부극', '카우보이', '상징적', '의상', '애초', '건조', '황무지', '방진', '방풍', '목적', '코트', '동', '동절기', '절기', '간', '간절기', '절기', '착용', '플', '코트', '벨기에', '플', '지방', '유래', '모양', '뿔', '뿔단추토', '단추', '토', '고리', '코트', '뿔', '뿔단추', '단추', '모양', '때문', '일명', '떡볶이', '코트', '랩', '코트', '외래어', '표기법', '랩', '코트', '표기', '두가', '두가지가', '지가', '하나', '연구소', '코트', '코트실험복', '실험', '복', '의사', '가운', '가운', '의미', '화학', '화학물질', '물질', '각종', '오염', '환경', '흰색', '청결', '이미지', '동시', '오염', '여부', '파악', '패션', '아이템', '작업복', '하나', '의미', '디자인', '전면', '전면부', '부', '깃', '특징', '매킨토시', '코트', '맥', '맥코트', '코트', '년', '스코', '스코틀랜드', '틀', '랜드', '디자인', '코트', '허리끈', '컬러', '디테일', '심플', '레인', '코트', '본래', '매킨토시', '원단', '이', '이름', '이젠', '원단', '특유', '디자인', '코트', '전체', '맥', '맥코트', '코트', '인', '코트', '기장', '소매', '오버', '오버코트', '코트', '본래', '스코', '스코틀랜드', '틀', 

In [11]:
# Freq Cutting
processed_counter = Counter(processed_words)
processed_words = [word for word in processed_words if processed_counter[word] > 3]

## Data Preprocessing - Train

In [12]:
word_counter = Counter(processed_words)
total_words = sum(word_counter.values())

In [13]:
vocab_to_int = {word[0]: i for i, word in enumerate(word_counter.most_common())}
int_to_vocab = {item[1]: item[0] for item in vocab_to_int.items()}
int_words = [vocab_to_int[word] for word in processed_words]

In [14]:
# Subsampling
def ignore_prob(freq, thres=1e-5):
    prob = 1 - np.sqrt(thres / freq)
    return prob > rand.random()

In [15]:
train_words = [word for word in processed_words if not ignore_prob(word_counter[word]/total_words)]

In [16]:
train_words

['업',
 '코트',
 '토',
 '모양',
 '랩',
 '오염',
 '스코틀랜드',
 '컬러',
 '원단',
 '맥',
 '소매',
 '오버',
 '코트',
 '체스터',
 '백작',
 '라펠',
 '필드',
 '일상',
 '상징',
 '패딩',
 '전기',
 '가면라이더',
 '가면라이더',
 '라이더',
 '가면',
 '가면',
 '게',
 '기사',
 '로',
 '대',
 '하라',
 '데',
 '데이비드',
 '강',
 '쥬',
 '현',
 '회색도시',
 '란',
 '레',
 '홈',
 '셜',
 '누',
 '전대',
 '다이',
 '해적',
 '렌',
 '시리즈',
 '렉',
 '에미',
 '필',
 '처',
 '오리',
 '해군',
 '턴',
 '간지',
 '왕',
 '왕',
 '이치',
 '닥터',
 '때',
 '오',
 '야마',
 '이드',
 '둘',
 '하라',
 '주요',
 '레드',
 '모습',
 '프리',
 '아웃',
 '레인저',
 '더블',
 '코트',
 '스포티',
 '불편',
 '뉴',
 '신',
 '발란스',
 '상징',
 '도',
 '이상',
 '라인',
 '경우',
 '경우',
 '노력',
 '유희',
 '상표',
 '사람',
 '운동화',
 '대상',
 '수상',
 '발',
 '만원',
 '출시',
 '패딩',
 '다운',
 '일부',
 '해외',
 '발란스',
 '재',
 '한국인',
 '발란스',
 '칼',
 '신발',
 '발',
 '발란스',
 '시리즈',
 '쪽',
 '런',
 '리스트',
 '발란스',
 '선수',
 '뉴발란스',
 '뉴',
 '자회사',
 '키트',
 '제작',
 '시티',
 '대표팀',
 '스타',
 '국가대표팀',
 '대표팀',
 '안',
 '키',
 '법',
 '칸',
 '쇼',
 '도',
 '닥터',
 '노',
 '단간',
 '듀',
 '마사',
 '레이',
 '유우',
 '결정',
 '더',
 '초',
 '전자',
 '전대',
 '저',
 '메',
 '듀티',
 '런',
 '재판',

In [17]:
def get_target(words, idx, window_size=5, to_int=False):
    rng = np.random.randint(1, window_size+1)
    
    start = idx - rng if idx - rng > 0 else 0  #*
    end = idx + rng
    
    selected = words[start:end+1]
    return selected if not to_int else [vocab_to_int[word] for word in selected]

In [18]:
def get_batches(words, batch_size, window_size=5):
    n_batches = len(words)//batch_size
    
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = vocab_to_int[batch[ii]]
            batch_y = get_target(batch, ii, window_size, to_int=True)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [19]:
class BatchGenerator(object):
    def __init__(self, data, batch_size):
        self.batch_size = batch_size
        
        self.data = data
        self.data_length = len(self.data)
        
        self.n_batches = (self.data_length//self.batch_size)
        if self.data_length%self.batch_size != 0: self.n_batches += 1
        
        self.current_batch = 0
        
    def next_batch(self):
        start_idx = self.batch_size*self.current_batch
        
        batch = self.data[start_idx:start_idx+self.batch_size]
        self.current_batch += 1
        
        if self.current_batch*self.batch_size >= self.data_length:
            self.current_batch = 0
            
        return batch
    
    def reset(self):
        self.current_batch = 0

In [20]:
# Save Vocab_to_Int, Int_to_Vocab
%mkdir saves

mkdir: `saves' 디렉토리를 만들 수 없습니다: 파일이 있습니다


In [21]:
vocab_to_int_path = './saves/vti.txt'
int_to_vocab_path = './saves/itv.txt'

In [22]:
vti_file = open(vocab_to_int_path, 'w')
itv_file = open(int_to_vocab_path, 'w')

In [23]:
for item in vocab_to_int.items():
    vti_file.write('{},{}\n'.format(item[0], item[1]))
vti_file.close()

In [24]:
for item in int_to_vocab.items():
    itv_file.write('{},{}\n'.format(item[0], item[1]))
itv_file.close()

## Model Building

In [25]:
invader_net = tf.Graph()

In [26]:
with invader_net.as_default():
    inputs_ = tf.placeholder(tf.int32, shape=[None], name='inputs')
    targets_ = tf.placeholder(tf.int32, shape=[None, None], name='targets')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    
    batch_array_ = tf.placeholder(tf.float32, [None, None], name='batch_array')
    k_ = tf.placeholder(tf.int32, name='k')

In [27]:
n_vocab = len(int_to_vocab)
n_embedding = 250
with invader_net.as_default():
    embedding = tf.Variable(tf.random_uniform([n_vocab, n_embedding],-1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [28]:
n_sampled = 150
with invader_net.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))

    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, targets_, embed, n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_).minimize(cost)
    
    saver = tf.train.Saver()

In [29]:
with invader_net.as_default():
    normed_embedding = tf.nn.l2_normalize(embedding, dim=1)  # embed
    normed_array = tf.nn.l2_normalize(batch_array_, dim=1)
    
    cosine_similarity = tf.matmul(normed_array, tf.transpose(normed_embedding, [1, 0]))
    #closest_words = tf.argmax(cosine_similarity, 1)
    closest_words = tf.nn.top_k(cosine_similarity, k_)

In [30]:
with invader_net.as_default():
    valid_size = 16
    valid_window = 100

    valid_examples = np.array(rand.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               rand.sample(range(0,valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

## Model Training

In [31]:
# Hyperparameters
n_epochs = 2000
learning_rate = 0.01
batch_size = 200
window_size = 10

In [32]:
%mkdir checkpoints
save_path = './checkpoints/invader_net.ckpt'

mkdir: `checkpoints' 디렉토리를 만들 수 없습니다: 파일이 있습니다


In [33]:
train_data = [(vocab_to_int[word], get_target(train_words, i, window_size=window_size, to_int=True)) for i, word in enumerate(train_words)]

In [34]:
batch_gen = BatchGenerator(train_data, batch_size)

In [35]:
batch_gen.reset()

In [36]:
with tf.Session(graph=invader_net) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        batches = get_batches(train_words, batch_size, window_size)
        for x, y in batches: 
            feed = {
                inputs_: np.array(x),
                targets_: np.array(y)[:, None],
                learning_rate_: learning_rate
            }
            
            curr_cost, _ = sess.run([cost, optimizer], feed_dict=feed)
        print('Epoch {} / {} Complete. Current Cost : {}'.format(epoch+1, n_epochs, curr_cost))
    
    print('Training Complete.. Saving..')
    saver.save(sess, save_path)
    print('Saving Complete.')

Epoch 1 / 2000 Complete. Current Cost : 5.304858207702637
Epoch 2 / 2000 Complete. Current Cost : 4.693203449249268
Epoch 3 / 2000 Complete. Current Cost : 3.9596903324127197
Epoch 4 / 2000 Complete. Current Cost : 3.513970136642456
Epoch 5 / 2000 Complete. Current Cost : 3.1454970836639404
Epoch 6 / 2000 Complete. Current Cost : 2.937560558319092
Epoch 7 / 2000 Complete. Current Cost : 2.7205748558044434
Epoch 8 / 2000 Complete. Current Cost : 2.459038734436035
Epoch 9 / 2000 Complete. Current Cost : 2.9026296138763428
Epoch 10 / 2000 Complete. Current Cost : 2.341266632080078
Epoch 11 / 2000 Complete. Current Cost : 2.1237316131591797
Epoch 12 / 2000 Complete. Current Cost : 2.5355336666107178
Epoch 13 / 2000 Complete. Current Cost : 2.425962209701538
Epoch 14 / 2000 Complete. Current Cost : 2.388704776763916
Epoch 15 / 2000 Complete. Current Cost : 2.6928720474243164
Epoch 16 / 2000 Complete. Current Cost : 2.441187858581543
Epoch 17 / 2000 Complete. Current Cost : 2.078075647354126

Epoch 141 / 2000 Complete. Current Cost : 2.0572545528411865
Epoch 142 / 2000 Complete. Current Cost : 2.283604621887207
Epoch 143 / 2000 Complete. Current Cost : 1.9483966827392578
Epoch 144 / 2000 Complete. Current Cost : 1.7844936847686768
Epoch 145 / 2000 Complete. Current Cost : 2.4250357151031494
Epoch 146 / 2000 Complete. Current Cost : 1.8029170036315918
Epoch 147 / 2000 Complete. Current Cost : 1.8136894702911377
Epoch 148 / 2000 Complete. Current Cost : 2.3494138717651367
Epoch 149 / 2000 Complete. Current Cost : 1.705968976020813
Epoch 150 / 2000 Complete. Current Cost : 2.4300477504730225
Epoch 151 / 2000 Complete. Current Cost : 2.157177448272705
Epoch 152 / 2000 Complete. Current Cost : 2.0951106548309326
Epoch 153 / 2000 Complete. Current Cost : 2.037151575088501
Epoch 154 / 2000 Complete. Current Cost : 2.1058783531188965
Epoch 155 / 2000 Complete. Current Cost : 2.2186906337738037
Epoch 156 / 2000 Complete. Current Cost : 2.255744695663452
Epoch 157 / 2000 Complete. Cu

Epoch 277 / 2000 Complete. Current Cost : 2.0244333744049072
Epoch 278 / 2000 Complete. Current Cost : 1.9047026634216309
Epoch 279 / 2000 Complete. Current Cost : 1.6594123840332031
Epoch 280 / 2000 Complete. Current Cost : 1.9686614274978638
Epoch 281 / 2000 Complete. Current Cost : 1.7037763595581055
Epoch 282 / 2000 Complete. Current Cost : 2.3290820121765137
Epoch 283 / 2000 Complete. Current Cost : 1.6779402494430542
Epoch 284 / 2000 Complete. Current Cost : 2.2239341735839844
Epoch 285 / 2000 Complete. Current Cost : 1.8915579319000244
Epoch 286 / 2000 Complete. Current Cost : 1.9744765758514404
Epoch 287 / 2000 Complete. Current Cost : 2.222569227218628
Epoch 288 / 2000 Complete. Current Cost : 1.8540372848510742
Epoch 289 / 2000 Complete. Current Cost : 2.1693644523620605
Epoch 290 / 2000 Complete. Current Cost : 1.9837397336959839
Epoch 291 / 2000 Complete. Current Cost : 1.7155988216400146
Epoch 292 / 2000 Complete. Current Cost : 1.81565260887146
Epoch 293 / 2000 Complete. 

Epoch 413 / 2000 Complete. Current Cost : 1.9600881338119507
Epoch 414 / 2000 Complete. Current Cost : 1.826133370399475
Epoch 415 / 2000 Complete. Current Cost : 2.148181200027466
Epoch 416 / 2000 Complete. Current Cost : 1.8130611181259155
Epoch 417 / 2000 Complete. Current Cost : 1.9793277978897095
Epoch 418 / 2000 Complete. Current Cost : 1.9873919486999512
Epoch 419 / 2000 Complete. Current Cost : 1.8346232175827026
Epoch 420 / 2000 Complete. Current Cost : 2.360318899154663
Epoch 421 / 2000 Complete. Current Cost : 2.079741954803467
Epoch 422 / 2000 Complete. Current Cost : 2.1221516132354736
Epoch 423 / 2000 Complete. Current Cost : 1.5748573541641235
Epoch 424 / 2000 Complete. Current Cost : 1.9934096336364746
Epoch 425 / 2000 Complete. Current Cost : 1.7000194787979126
Epoch 426 / 2000 Complete. Current Cost : 2.1072278022766113
Epoch 427 / 2000 Complete. Current Cost : 1.7671104669570923
Epoch 428 / 2000 Complete. Current Cost : 1.7099987268447876
Epoch 429 / 2000 Complete. C

Epoch 549 / 2000 Complete. Current Cost : 2.1897342205047607
Epoch 550 / 2000 Complete. Current Cost : 2.1173503398895264
Epoch 551 / 2000 Complete. Current Cost : 1.6699204444885254
Epoch 552 / 2000 Complete. Current Cost : 2.174483060836792
Epoch 553 / 2000 Complete. Current Cost : 2.0539472103118896
Epoch 554 / 2000 Complete. Current Cost : 2.017896890640259
Epoch 555 / 2000 Complete. Current Cost : 1.7385958433151245
Epoch 556 / 2000 Complete. Current Cost : 1.7345186471939087
Epoch 557 / 2000 Complete. Current Cost : 1.7950304746627808
Epoch 558 / 2000 Complete. Current Cost : 2.0615949630737305
Epoch 559 / 2000 Complete. Current Cost : 1.7186380624771118
Epoch 560 / 2000 Complete. Current Cost : 1.9157987833023071
Epoch 561 / 2000 Complete. Current Cost : 1.9523714780807495
Epoch 562 / 2000 Complete. Current Cost : 1.6121281385421753
Epoch 563 / 2000 Complete. Current Cost : 1.7819730043411255
Epoch 564 / 2000 Complete. Current Cost : 1.611220359802246
Epoch 565 / 2000 Complete. 

...

Epoch 1365 / 2000 Complete. Current Cost : 1.6814686059951782
Epoch 1366 / 2000 Complete. Current Cost : 2.0499114990234375
Epoch 1367 / 2000 Complete. Current Cost : 1.8590632677078247
Epoch 1368 / 2000 Complete. Current Cost : 1.622239589691162
Epoch 1369 / 2000 Complete. Current Cost : 1.9048181772232056
Epoch 1370 / 2000 Complete. Current Cost : 1.9820235967636108
Epoch 1371 / 2000 Complete. Current Cost : 1.5857528448104858
Epoch 1372 / 2000 Complete. Current Cost : 1.5595306158065796
Epoch 1373 / 2000 Complete. Current Cost : 1.7528671026229858
Epoch 1374 / 2000 Complete. Current Cost : 1.749764323234558
Epoch 1375 / 2000 Complete. Current Cost : 1.7137576341629028
Epoch 1376 / 2000 Complete. Current Cost : 1.8520351648330688
Epoch 1377 / 2000 Complete. Current Cost : 1.766675591468811
Epoch 1378 / 2000 Complete. Current Cost : 1.955838680267334
Epoch 1379 / 2000 Complete. Current Cost : 1.6749963760375977
Epoch 1380 / 2000 Complete. Current Cost : 1.6708178520202637
Epoch 1381 /

Epoch 1501 / 2000 Complete. Current Cost : 1.8598204851150513
Epoch 1502 / 2000 Complete. Current Cost : 1.6787608861923218
Epoch 1503 / 2000 Complete. Current Cost : 1.4323393106460571
Epoch 1504 / 2000 Complete. Current Cost : 1.8503198623657227
Epoch 1505 / 2000 Complete. Current Cost : 1.5651124715805054
Epoch 1506 / 2000 Complete. Current Cost : 1.737226128578186
Epoch 1507 / 2000 Complete. Current Cost : 1.4930171966552734
Epoch 1508 / 2000 Complete. Current Cost : 1.4442753791809082
Epoch 1509 / 2000 Complete. Current Cost : 1.619733452796936
Epoch 1510 / 2000 Complete. Current Cost : 1.6295440196990967
Epoch 1511 / 2000 Complete. Current Cost : 1.7119144201278687
Epoch 1512 / 2000 Complete. Current Cost : 1.642303705215454
Epoch 1513 / 2000 Complete. Current Cost : 2.2265918254852295
Epoch 1514 / 2000 Complete. Current Cost : 1.5347360372543335
Epoch 1515 / 2000 Complete. Current Cost : 1.8051369190216064
Epoch 1516 / 2000 Complete. Current Cost : 1.6520570516586304
Epoch 1517 

Epoch 1637 / 2000 Complete. Current Cost : 1.5574676990509033
Epoch 1638 / 2000 Complete. Current Cost : 1.8249484300613403
Epoch 1639 / 2000 Complete. Current Cost : 1.405877709388733
Epoch 1640 / 2000 Complete. Current Cost : 1.893988847732544
Epoch 1641 / 2000 Complete. Current Cost : 1.8936794996261597
Epoch 1642 / 2000 Complete. Current Cost : 1.56336510181427
Epoch 1643 / 2000 Complete. Current Cost : 1.5195446014404297
Epoch 1644 / 2000 Complete. Current Cost : 1.7443469762802124
Epoch 1645 / 2000 Complete. Current Cost : 1.5227278470993042
Epoch 1646 / 2000 Complete. Current Cost : 1.691874384880066
Epoch 1647 / 2000 Complete. Current Cost : 2.294664144515991
Epoch 1648 / 2000 Complete. Current Cost : 1.6808804273605347
Epoch 1649 / 2000 Complete. Current Cost : 1.8661534786224365
Epoch 1650 / 2000 Complete. Current Cost : 1.595343828201294
Epoch 1651 / 2000 Complete. Current Cost : 1.4652438163757324
Epoch 1652 / 2000 Complete. Current Cost : 1.3846536874771118
Epoch 1653 / 20

Epoch 1773 / 2000 Complete. Current Cost : 1.8114153146743774
Epoch 1774 / 2000 Complete. Current Cost : 1.7179186344146729
Epoch 1775 / 2000 Complete. Current Cost : 1.4823884963989258
Epoch 1776 / 2000 Complete. Current Cost : 1.6021920442581177
Epoch 1777 / 2000 Complete. Current Cost : 1.552770733833313
Epoch 1778 / 2000 Complete. Current Cost : 2.033515691757202
Epoch 1779 / 2000 Complete. Current Cost : 1.9267094135284424
Epoch 1780 / 2000 Complete. Current Cost : 1.692441701889038
Epoch 1781 / 2000 Complete. Current Cost : 1.8428375720977783
Epoch 1782 / 2000 Complete. Current Cost : 1.9759187698364258
Epoch 1783 / 2000 Complete. Current Cost : 1.8230904340744019
Epoch 1784 / 2000 Complete. Current Cost : 1.794090986251831
Epoch 1785 / 2000 Complete. Current Cost : 1.8114426136016846
Epoch 1786 / 2000 Complete. Current Cost : 1.800541877746582
Epoch 1787 / 2000 Complete. Current Cost : 1.5534266233444214
Epoch 1788 / 2000 Complete. Current Cost : 1.8307009935379028
Epoch 1789 / 

Epoch 1909 / 2000 Complete. Current Cost : 1.745316505432129
Epoch 1910 / 2000 Complete. Current Cost : 1.9876846075057983
Epoch 1911 / 2000 Complete. Current Cost : 1.7969634532928467
Epoch 1912 / 2000 Complete. Current Cost : 1.9786919355392456
Epoch 1913 / 2000 Complete. Current Cost : 1.7936322689056396
Epoch 1914 / 2000 Complete. Current Cost : 1.7050634622573853
Epoch 1915 / 2000 Complete. Current Cost : 1.5872325897216797
Epoch 1916 / 2000 Complete. Current Cost : 1.7461987733840942
Epoch 1917 / 2000 Complete. Current Cost : 1.4205344915390015
Epoch 1918 / 2000 Complete. Current Cost : 1.4742687940597534
Epoch 1919 / 2000 Complete. Current Cost : 1.8264620304107666
Epoch 1920 / 2000 Complete. Current Cost : 1.6460193395614624
Epoch 1921 / 2000 Complete. Current Cost : 1.8322969675064087
Epoch 1922 / 2000 Complete. Current Cost : 1.4089733362197876
Epoch 1923 / 2000 Complete. Current Cost : 1.8055182695388794
Epoch 1924 / 2000 Complete. Current Cost : 1.489984393119812
Epoch 1925

## Testing Network

In [40]:
with tf.Session(graph=invader_net) as sess:
    saver.restore(sess, save_path)
    
    test_word = vocab_to_int['아디다스']
    test_embed = sess.run(embed, feed_dict={inputs_: [test_word]})
    
    feed = {
        batch_array_: test_embed,
        k_: 10
    }
    
    output = sess.run(closest_words, feed_dict=feed)
    vocab_outputs = [int_to_vocab[word] for word in output[1][0]]
    print(vocab_outputs)

INFO:tensorflow:Restoring parameters from ./checkpoints/invader_net.ckpt
['아디다스', '스니커즈', '비', '주력', '색깔', '결국', '셀', '한국', '주자', '분야']


In [38]:
with tf.Session(graph=invader_net) as sess:
    saver.restore(sess, save_path)
    sim = similarity.eval() ###
    for i in range(valid_size):
        valid_word = int_to_vocab[valid_examples[i]] 
        top_k = 8 # number of nearest neighbors ###
        nearest = (-sim[i, :]).argsort()[1:top_k+1] ###
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
            close_word = int_to_vocab[nearest[k]]
            log = '%s %s,' % (log, close_word)
        print(log)

INFO:tensorflow:Restoring parameters from ./checkpoints/invader_net.ckpt
Nearest to 라인: 체형, 불어, 방향, 채택, 제안, 체크무늬, 기존, 가슴,
Nearest to 르: 제법, 위지, 옆, 기원, 상대적, 이탈리아, 아웃솔, 생각,
Nearest to 때: 오, 야마, 간지, 주요, 왕, 턴, 닥터, 레드,
Nearest to 로: 강, 파란색, 네이비, 다, 패, 현, 초, 대,
Nearest to 마: 워크, 고, 요원, 조커, 아, 쿠로, 지, 가이,
Nearest to 다양: 링, 에드워드, 설립, 독일, 하이, 가지, 그린, 끝,
Nearest to 미국: 컨트리, 밖, 피트, 노출, 리복의, 피, 로우, 발표,
Nearest to 라: 민속복, 색조, 아몬드, 다이아몬드, 백색, 무늬, 센스, 다이,
Nearest to 구두: 질, 이야기, 일본어, 어원, 차이, 급, 여성용, 원의,
Nearest to 테일러: 생, 감, 올스타, 동일, 패치, 트로피컬, 강조, 타운,
Nearest to 사용: 중성, 아메리칸, 세제, 피셔, 사람, 슬리브, 뒤, 전략,
Nearest to 여성: 웨스턴, 닉, 아시아, 앤드, 남성, 지금, 오늘날, 알로하,
Nearest to 버스: 앤, 제조, 판매, 창립, 독립, 베스트, 진출, 레이션,
Nearest to 의미: 단조, 감각, 신사복, 양말, 의, 스타일, 벌, 수,
Nearest to 경우: 노력, 물론, 상표, 대상, 도, 신, 수상, 아래,
Nearest to 팬츠: 스쿨, 체크무늬, 수렵, 여학생, 방향, 화, 타이, 체형,
